In [1]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.spark.processing import PySparkProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
import sagemaker

In [6]:
sagemaker.Session().boto_region_name

'us-east-1'

In [2]:
bucket = sagemaker.Session().default_bucket()

In [6]:
pyspark_processor = PySparkProcessor(
    framework_version="2.4",
    role='arn:aws:iam::111111111111:role/service-role/AmazonSageMaker-ExecutionRole-20200101T000001',
    instance_type="local",
    instance_count=1,
    base_job_name="data-validation"
)

In [ ]:
pyspark_processor.run(
    submit_app="./preprocessing/data_validation.py",
    arguments=["--s3_input_data", "s3://mlops1/dados_fraude.tsv",
               "--s3_output_metrics_repository", f"s3://{bucket}/data_validation/repository/metrics.json"],
    outputs=[ProcessingOutput(output_name="suggested_constraints", source="/opt/ml/processing/output/suggestions")]    
)

In [4]:
sklearn_processor = SKLearnProcessor(
    framework_version="0.23-1",
    role='arn:aws:iam::111111111111:role/service-role/AmazonSageMaker-ExecutionRole-20200101T000001',
    instance_type="local",
    instance_count=1,
    base_job_name="featuring_engineering"
)

In [8]:
sklearn_processor.run(
    code="./preprocessing/feature_engineering.py",
    inputs=[
        ProcessingInput(source="s3://mlops1/dados_fraude.tsv", destination="/opt/ml/processing/input")
    ],
    outputs=[
        ProcessingOutput(output_name="train_data", source="/opt/ml/processing/output/train"),
        ProcessingOutput(output_name="dev_data", source="/opt/ml/processing/output/dev"),
        ProcessingOutput(output_name="test_data", source="/opt/ml/processing/output/test"),
        ProcessingOutput(output_name="features_info", source="/opt/ml/processing/output/features"),
        ProcessingOutput(output_name="text_model", source="/opt/ml/processing/output/text_model")
    ],
    arguments=["--train_size", "0.8"]
)


Job Name:  featuring_engineering-2021-12-26-04-04-48-432
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://mlops1/dados_fraude.tsv', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-681301531115/featuring_engineering-2021-12-26-04-04-48-432/input/code/feature_engineering.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'train_data', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-681301531115/featuring_engineering-2021-12-26-04-04-48-432/output/train_data', 'LocalPath': '/opt/ml/processing/output/train', 'S3UploadMode': 'EndOfJob'}}, {'OutputName': 'dev_data', 'AppManaged': Fals

RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmpuw8zrmyl/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit']